# Aula 08 - Aprendendo a Ranquear - Exercícios

In [1]:
import pandas as pd
import numpy as np
import random

### Importar base de dados

In [2]:
import wget
!python3 -m wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-compact.tar.gz
!tar -xvzf ml-20m-compact.tar.gz

100% [....................................................] 65019041 / 65019041
Saved under ml-20m-compact.tar.gz
100% [....................................................] 65019041 / 65019041
Saved under ml-20m-compact.tar.gz
dataset/
dataset/tags_sample.csv
dataset/._.DS_Store
dataset/.DS_Store
dataset/movies_sample.csv
dataset/._genome-tags.csv
dataset/genome-tags.csv
dataset/._ml-youtube.csv
dataset/ml-youtube.csv
dataset/._genome-scores.csv
dataset/genome-scores.csv
dataset/
dataset/tags_sample.csv
dataset/._.DS_Store
dataset/.DS_Store
dataset/movies_sample.csv
dataset/._genome-tags.csv
dataset/genome-tags.csv
dataset/._ml-youtube.csv
dataset/ml-youtube.csv
dataset/._genome-scores.csv
dataset/genome-scores.csv
dataset/ratings_sample.csv
dataset/ratings_sample.csv


In [3]:
movies = pd.read_csv('./dataset/movies_sample.csv', names=['itemId', 'title', 'genre'], header=0)
ratings = pd.read_csv('./dataset/ratings_sample.csv', names=['userId', 'itemId', 'rating', 'timestamp'], header=0)
df = ratings[['userId', 'itemId', 'rating']]
df = df.merge(movies[['itemId', 'title']])
df

,userId,itemId,rating,title
0,11,7481,5.0,Enemy Mine (1985)
1,11,1046,4.5,Beautiful Thing (1996)
2,11,616,4.0,"Aristocats, The (1970)"
3,11,3535,2.0,American Psycho (2000)
4,11,5669,5.0,Bowling for Columbine (2002)
...,...,...,...,...
190616,138493,288,5.0,Natural Born Killers (1994)
190617,138493,1748,5.0,Dark City (1998)
190618,138493,616,4.0,"Aristocats, The (1970)"
190619,138493,1597,4.5,Conspiracy Theory (1997)


### Mapeamento de ids

In [4]:
map_users = {user: idx for idx, user in enumerate(df.userId.unique())}
map_items = {item: idx for idx, item in enumerate(df.itemId.unique())}
df['userId'] = df['userId'].map(map_users)
df['itemId'] = df['itemId'].map(map_items)
map_title = {}

for _, row in df.iterrows():
    map_title[row.itemId] = row.title


### Divisão da base em treino e teste

In [5]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=.2, random_state=2)

### Nesta aula, você podera escolher entre implementar o primeiro, o segundo ou ambos os exercícios abaixo.

***Exercício 01:*** Comparação entre um algoritmo pair-wise e point-wise:
- Utilizando o BPR implementado em aula, gere uma lista de 10 recomendações para um determinado usuário da base.
- Calcule a precisão média dessas recomendações usando como ground-truth o conjunto de teste. 
- Utilize o algoritmo MatrixFactorization (SVD otimizado) de predição de notas para prever as notas dos itens que aquele usuário não avaliou ainda. 
- Ordene inversamente esses itens pela nota predita, e recomende os 10 primeiros filmes.
- Calcule a precisão média dessas recomendações e compare o resultado com o resultado obtido pelo BPR. 

In [6]:
# ---  BPR (pair-wise) vs MF (point-wise) ---
from collections import defaultdict, Counter
import math

np.random.seed(42)
random.seed(42)

n_users = df['userId'].nunique()
n_items = df['itemId'].nunique()
eligible_users = [u for u in train['userId'].unique() if u in set(test['userId'].unique())]
target_user = max(eligible_users, key=lambda u: len(test[test['userId'] == u]), default=train['userId'].iloc[0])
print(f'Usuário alvo para comparação: {target_user}')

user_positive_items = train.groupby('userId')['itemId'].apply(set).to_dict()
all_items = np.arange(n_items)

movies_mapped = movies[movies['itemId'].isin(map_items.keys())].copy()
movies_mapped['itemId'] = movies_mapped['itemId'].map(map_items)
item_genre_map = {row.itemId: set((row.genre or '').split('|')) for _, row in movies_mapped.iterrows()}

def precision_at_k(recommendations: list[int], user_id: int, k: int = 10) -> float:
    """Calculo a precisão@k usando o conjunto de teste como ground truth."""
    relevant_items = set(test[test['userId'] == user_id]['itemId'])
    if not relevant_items:
        return float('nan')
    hits = sum(1 for item in recommendations[:k] if item in relevant_items)
    return hits / k

class SimpleBPR:
    def __init__(self, n_users: int, n_items: int, n_factors: int = 40, learning_rate: float = 0.05,
                 reg: float = 0.002, sampler=None):
        self.n_users = n_users
        self.n_items = n_items
        self.n_factors = n_factors
        self.lr = learning_rate
        self.reg = reg
        self.sampler = sampler
        self.user_factors = 0.01 * np.random.randn(n_users, n_factors)
        self.item_factors = 0.01 * np.random.randn(n_items, n_factors)

    def fit(self, interactions: pd.DataFrame, epochs: int = 15, samples_per_epoch: int | None = None):
        user_groups = interactions.groupby('userId')['itemId'].apply(set)
        self.user_pos = {u: set(items) for u, items in user_groups.items()}
        self.users_with_data = list(self.user_pos.keys())
        if samples_per_epoch is None:
            samples_per_epoch = len(interactions)
        for epoch in range(epochs):
            for _ in range(samples_per_epoch):
                user = random.choice(self.users_with_data)
                pos_item, neg_item = self._draw_items(user)
                self._update_factors(user, pos_item, neg_item)

    def _draw_items(self, user: int) -> tuple[int, int]:
        if self.sampler:
            return self.sampler(user, self.user_pos, all_items)
        pos_item = random.choice(tuple(self.user_pos[user]))
        neg_candidates = list(set(all_items) - self.user_pos[user])
        neg_item = random.choice(neg_candidates) if neg_candidates else random.choice(all_items)
        return pos_item, neg_item

    def _update_factors(self, user: int, pos_item: int, neg_item: int) -> None:
        user_vec = self.user_factors[user]
        pos_vec = self.item_factors[pos_item]
        neg_vec = self.item_factors[neg_item]
        x_uij = np.dot(user_vec, pos_vec - neg_vec)
        sigmoid = 1 / (1 + np.exp(-x_uij))
        grad_user = sigmoid * (pos_vec - neg_vec) - self.reg * user_vec
        grad_pos = sigmoid * user_vec - self.reg * pos_vec
        grad_neg = -sigmoid * user_vec - self.reg * neg_vec
        self.user_factors[user] += self.lr * grad_user
        self.item_factors[pos_item] += self.lr * grad_pos
        self.item_factors[neg_item] += self.lr * grad_neg

    def recommend(self, user: int, top_k: int = 10) -> list[int]:
        scores = self.user_factors[user] @ self.item_factors.T
        seen_items = self.user_pos.get(user, set())
        scores[list(seen_items)] = -np.inf

        top_items = np.argsort(scores)[::-1][:top_k]
        return top_items.tolist()

class SimpleMF:
    def __init__(self, n_users: int, n_items: int, n_factors: int = 40, learning_rate: float = 0.01, reg: float = 0.02):
        self.n_users = n_users
        self.n_items = n_items
        self.n_factors = n_factors
        self.lr = learning_rate
        self.reg = reg
        self.global_mean = ratings['rating'].mean()
        self.user_factors = 0.01 * np.random.randn(n_users, n_factors)
        self.item_factors = 0.01 * np.random.randn(n_items, n_factors)
        self.user_bias = np.zeros(n_users)
        self.item_bias = np.zeros(n_items)

    def fit(self, ratings_df: pd.DataFrame, epochs: int = 20):
        for epoch in range(epochs):
            shuffled = ratings_df.sample(frac=1, random_state=epoch)
            for row in shuffled.itertuples(index=False):
                u, i, rating = row
                pred = self.predict_single(u, i)
                err = rating - pred
                self.user_bias[u] += self.lr * (err - self.reg * self.user_bias[u])
                self.item_bias[i] += self.lr * (err - self.reg * self.item_bias[i])
                user_vec = self.user_factors[u]
                item_vec = self.item_factors[i]
                self.user_factors[u] += self.lr * (err * item_vec - self.reg * user_vec)
                self.item_factors[i] += self.lr * (err * user_vec - self.reg * item_vec)

    def predict_single(self, user: int, item: int) -> float:
        return (self.global_mean + self.user_bias[user] + self.item_bias[item] +
                self.user_factors[user] @ self.item_factors[item])

    def recommend(self, user: int, known_items: set[int], top_k: int = 10) -> list[int]:
        scores = (self.global_mean + self.user_bias[user] + self.item_bias +
                  self.user_factors[user] @ self.item_factors.T)
        mask = np.full_like(scores, False, dtype=bool)
        mask[list(known_items)] = True
        scores[mask] = -np.inf
        return np.argsort(scores)[::-1][:top_k].tolist()

bpr_model = SimpleBPR(n_users, n_items)
bpr_model.fit(train, epochs=15, samples_per_epoch=5000)
bpr_recommendations = bpr_model.recommend(target_user, top_k=10)
precision_bpr = precision_at_k(bpr_recommendations, target_user)

mf_model = SimpleMF(n_users, n_items)
mf_model.fit(train[['userId', 'itemId', 'rating']], epochs=20)
known_items_target = user_positive_items.get(target_user, set())
mf_recommendations = mf_model.recommend(target_user, known_items_target, top_k=10)
precision_mf = precision_at_k(mf_recommendations, target_user)

def attach_titles(item_list: list[int]) -> pd.DataFrame:
    return pd.DataFrame({
        'itemId': item_list,
        'title': [map_title.get(item, f'Item {item}') for item in item_list]
    })

print("\nRecomendações BPR (pair-wise):")
display(attach_titles(bpr_recommendations))
print(f"Precisão@10 BPR: {precision_bpr:.3f}")

print("\nRecomendações MF (point-wise):")
display(attach_titles(mf_recommendations))
print(f"Precisão@10 MF: {precision_mf:.3f}")

comparison_df = pd.DataFrame({
    'modelo': ['BPR pair-wise', 'Matrix Factorization point-wise'],
    'precision@10': [precision_bpr, precision_mf]
})
comparison_df

Usuário alvo para comparação: 9518

Recomendações BPR (pair-wise):

Recomendações BPR (pair-wise):


,itemId,title
0,18,Life Is Beautiful (La Vita è bella) (1997)
1,366,Class of 1999 (1990)
2,302,Rudderless (2014)
3,412,"Toughest Man in the World, The (1984)"
4,324,Booker's Place: A Mississippi Story (2012)
5,407,Christmas Comes but Once a Year (1936)
6,177,Emma (1996)
7,308,"Dinosaur Project, The (2012)"
8,374,Raspberry Boat Refugee (2014)
9,304,Hara-Kiri: Death of a Samurai (2011)


Precisão@10 BPR: 0.100

Recomendações MF (point-wise):


,itemId,title
0,295,Love Streams (1984)
1,18,Life Is Beautiful (La Vita è bella) (1997)
2,220,Hearts and Minds (1974)
3,222,"Earrings of Madame de..., The (Madame de...) (..."
4,165,"Ordet (Word, The) (1955)"
5,281,"Muriel, or The Time of Return (Muriel ou Le te..."
6,129,Interstellar (2014)
7,124,Into the Woods (1991)
8,253,Shut Up and Play the Hits (2012)
9,40,Manon of the Spring (Manon des sources) (1986)


Precisão@10 MF: 0.300


,modelo,precision@10
0,BPR pair-wise,0.1
1,Matrix Factorization point-wise,0.3


**Explicação**
- Treinei um BPR simples tratando o problema como pareamento usuário-item com amostras negativas aleatórias, evitando recomendar itens já vistos e ranqueando por `score = u·i`.
- Em paralelo treinei um MF point-wise (SVD otimizado) que minimiza o erro quadrático das notas e gera escores explícitos para todos os itens não vistos.
- Para um mesmo usuário-alvo calculei `precision@10` em cima do conjunto de teste e comparei as listas de recomendações e métricas para evidenciar as diferenças entre abordagens pair-wise e point-wise.

***Exercicio 02:*** Na aula vimos a implementação do método draw() do algoritmo BPR tradicional. Para um dado usuário, este método retorna aleatoriamente um item que ele viu (**item i**) e um outro item que ele não conhece (**item j**). O algoritmo assume que o item visto é preferível ao que ele não viu, e isso é usado para maximizar a diferença entre os scores desses itens (veja a variável **x_uij** da implementação). 

Um problema dessa abordagem é que os itens não vistos são necessariamente encarados como menos relevante pelo algoritmo, o que nem sempre acontece pois pode ser que o usuário não tenha interagido com aquele item pois não o conhece, mas que poderia gostar. Um outro problema é quando ambos os itens i e j foram vistos pelo usuário, o que numa estratégia com feedback exclusivamente implícito o BPR não consegue diferenciar qual item é preferível ao usuário. 

Você consegue pensar numa estratégia aperfeiçoada para o método draw()? Exemplos:
- Se tivermos acesso aos metadados, podemos construir um perfil para o usuário de modo que o método draw() vai retornar itens j que estão mais distantes desse perfil.
- Se dois itens i e j foram vistos, use os metadados (ou as notas, caso use feedback explícito) para decidir qual deles deve ser o i e qual deve ser o j. 
- Etc.

Implemente pelo menos uma estratégia de aperfeiçoamento do BPR, e compare com a versão original. 

In [7]:
# --- aperfeiçoando o draw() do BPR ---
def build_user_genre_profiles(user_item_sets: dict[int, set[int]]) -> dict[int, Counter]:
    """Monto o perfil de gêneros de cada usuário a partir das interações positivas."""
    profiles = {}
    for user, items in user_item_sets.items():
        genre_counter = Counter()
        for item in items:
            genre_counter.update(item_genre_map.get(item, set()))
        profiles[user] = genre_counter
    return profiles

user_genre_profiles = build_user_genre_profiles(user_positive_items)
all_items_set = set(all_items.tolist())

def make_genre_aware_sampler():
    """Crio um sampler que privilegia itens negativos com gêneros distantes do perfil do usuário."""
    def sampler(user: int, user_pos: dict[int, set[int]], _all_items: np.ndarray) -> tuple[int, int]:
        pos_item = random.choice(tuple(user_pos[user]))
        unseen_items = list(all_items_set - user_pos[user])
        if not unseen_items:
            unseen_items = list(all_items_set)
        pref_genres = set(user_genre_profiles.get(user, Counter()).keys())
        if pref_genres:
            weights = []
            for item in unseen_items:
                overlap = len(pref_genres & item_genre_map.get(item, set()))
                weights.append(1 / (1 + overlap))  # penalizo itens parecidos com o perfil
            total_weight = sum(weights)
            if total_weight > 0:
                neg_item = random.choices(unseen_items, weights=weights, k=1)[0]
            else:
                neg_item = random.choice(unseen_items)
        else:
            neg_item = random.choice(unseen_items)
        return pos_item, neg_item
    return sampler

genre_aware_sampler = make_genre_aware_sampler()

bpr_vanilla = SimpleBPR(n_users, n_items)
bpr_vanilla.fit(train, epochs=10, samples_per_epoch=4000)
vanilla_recs = bpr_vanilla.recommend(target_user, top_k=10)
precision_vanilla = precision_at_k(vanilla_recs, target_user)

bpr_genre = SimpleBPR(n_users, n_items, sampler=genre_aware_sampler)
bpr_genre.fit(train, epochs=10, samples_per_epoch=4000)
genre_recs = bpr_genre.recommend(target_user, top_k=10)
precision_genre = precision_at_k(genre_recs, target_user)

print("Recomendações BPR original:")
display(attach_titles(vanilla_recs))
print(f"Precisão@10 (original): {precision_vanilla:.3f}")

print("\nRecomendações BPR com sampler guiado por gênero:")
display(attach_titles(genre_recs))
print(f"Precisão@10 (aperfeiçoado): {precision_genre:.3f}")

pd.DataFrame({
    'versão': ['BPR original', 'BPR + sampler guiado'],
    'precision@10': [precision_vanilla, precision_genre]
})

Recomendações BPR original:


,itemId,title
0,249,Omar (2013)
1,413,Dots (1940)
2,388,How to Seduce Difficult Women (2009)
3,151,When in Rome (2010)
4,143,Boogeyman (2005)
5,415,97 Percent True (2008)
6,188,"Private Lives of Pippa Lee, The (2009)"
7,305,Australian Atomic Confessions (2005)
8,283,"ChubbChubbs!, The (2002)"
9,216,Until the Light Takes Us (2008)


Precisão@10 (original): 0.100

Recomendações BPR com sampler guiado por gênero:


,itemId,title
0,18,Life Is Beautiful (La Vita è bella) (1997)
1,263,Manito (2002)
2,238,I Think I Do (1997)
3,365,2AM: The Smiling Man (2013)
4,40,Manon of the Spring (Manon des sources) (1986)
5,374,Raspberry Boat Refugee (2014)
6,315,Castle of the Living Dead (Castello Dei Morti ...
7,241,Panic Button (2011)
8,134,Paranormal Activity 2 (2010)
9,298,Entre ses mains (2005)


Precisão@10 (aperfeiçoado): 0.200


,versão,precision@10
0,BPR original,0.1
1,BPR + sampler guiado,0.2


**Explicação**
- Extraí um perfil de gêneros para cada usuário e usei essa informação para guiar o `draw()`: escolho itens negativos com baixa sobreposição de gêneros, ou seja, exemplos realmente "distantes" do gosto atual.
- Treinei o BPR tradicional e o BPR com o novo sampler mantendo mesmos hiperparâmetros (épocas e taxa de aprendizado) para garantir comparação justa.
- Comparei listas e `precision@10`, mostrando como a seleção de amostras negativas mais informativas pode alterar o ranqueamento final.